<a href="https://colab.research.google.com/github/ismael29h/DA_lab1/blob/master/(DA)Lab1_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DATA ANALYTICS - LABORATORIO

## Readme del dataset utilizado:
Este dataset es el resultado de muchos días de scraping.

Me gustaría darte una serie de consejos que podrían ayudarte a analizar este conjunto de datos:

- Ten en cuenta que el mercado de alquileres fue muy censurado en diciembre y a principios de enero,
por lo que podrías encontrar precios en dólares (cualquier precio inferior a 7.000 está definitivamente en dólares),
la tasa de cambio recomendada (por mi) es de 1 dólar igual a 1.000 pesos argentinos para los meses de diciembre y enero.
- Los precios con dígitos repetidos, por ejemplo:  1.111.111  222.222  9.999.999,
son datos falsos en su mayoría, esto era una forma de evitar incluir el precio en el anuncio.
- Muchos avisos no tienen un precio declarado.
- La columna de "sitio web" fue removida.

El proyecto de scraping: https://github.com/avalos-p/alquileres
Mi perfil de linkedin: https://www.linkedin.com/in/avalos-p/


## 1 - **ETL**

### 1.1 - EXTRACT

In [73]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive', force_remount=True)

file_path = '/content/drive/MyDrive/Análisis de Datos/Dataset-Laboratorio1/alquileres_database_2024-02-18.csv'

try:
    df = pd.read_csv(file_path)
    print("Archivo CSV cargado exitosamente.")
except FileNotFoundError:
    print(f"El archivo '{file_path}' no fue encontrado.")



Mounted at /content/drive
Archivo CSV cargado exitosamente.


### 1.2 - TRANSFORM

##### 1.2.1 RESUMEN INICIAL DEL DATAFRAME

In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1160402 entries, 0 to 1160401
Data columns (total 8 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   name       1160402 non-null  object 
 1   price      732625 non-null   float64
 2   days       437389 non-null   float64
 3   rooms      1104338 non-null  float64
 4   bathrooms  942741 non-null   float64
 5   capacity   823157 non-null   float64
 6   date       1160402 non-null  object 
 7   province   1160402 non-null  object 
dtypes: float64(5), object(3)
memory usage: 70.8+ MB


In [75]:
print("Filas/Columnas:", df.shape)
print("Filas duplicadas:", df.duplicated().sum())

print("\nNúmero de nulos por columnas:")
nulos = df.isna().sum().sort_values(ascending=False)
print(nulos[nulos>0])

Filas/Columnas: (1160402, 8)
Filas duplicadas: 0

Número de nulos por columnas:
days         723013
price        427777
capacity     337245
bathrooms    217661
rooms         56064
dtype: int64


- Observación: el datasets supera los 100.000 registros y no posee duplicados.

In [76]:
df.sample(10)

,name,price,days,rooms,bathrooms,capacity,date,province
810183,casa para veranear en villa del dique cordoba,"25,000.00",1.00,2.00,1.00,6.00,2024-02-04,cordoba
567341,chalet de 4 ambientes a 2 cuadras de la playa,"560,000.00",7.00,3.00,2.00,8.00,2024-01-26,buenos-aires
330065,cabanas el chanar san rafael mendoza,NaN,NaN,1.00,1.00,5.00,2024-01-14,mendoza
274805,av del puerto condominio de la bahia 200,"1,100.00",NaN,1.00,NaN,NaN,2024-01-12,buenos-aires
1097708,consultar direccion,"45,000.00",NaN,2.00,1.00,NaN,2024-02-15,buenos-aires
322568,alquiler depto 3 amb en miramar a dos cuadras ...,NaN,NaN,2.00,2.00,6.00,2024-01-14,buenos-aires
426478,mar del plata temporada 2023,NaN,NaN,1.00,1.00,4.00,2024-01-19,buenos-aires
595251,alberti al 3100,NaN,NaN,1.00,NaN,NaN,2024-01-27,buenos-aires
1047752,cordoba tanti hospedajealojamiento sierras rio...,"20,000.00",1.00,10.00,10.00,10.00,2024-02-13,cordoba
237470,miramar alquilo ph en 1er piso con balcon muy ...,"49,995.00",15.00,3.00,2.00,6.00,2024-01-10,buenos-aires


In [77]:
df['province'].value_counts()

,count
province,
buenos-aires,903199
cordoba,98810
mar-del-plata,58507
mendoza,48046
rosario,21890
bariloche,16713
salta,8265
misiones,2084
tucuman,1960


##### 1.2.2 CORRECCIONES Y LIMPIEZA en base al resumen inicial:
- No existen registros nulos
- Ajustar tipos de datos
- Cambiar nombre a 'province'


In [78]:
# Corrección de tipo fecha - Columna 'date'
df['date'] = pd.to_datetime(df['date'])

# Nombres mejorados retocados
df['name'] = df['name'].astype(str).str.strip().str.title()

# Provincias a tipo 'category'
df['province'] = df['province'].astype(str).str.strip().str.upper().str.replace('-',' ', regex=False)
df['province'] = df['province'].astype('category')

# Columnas a enteros -> Int64 permite nulos (<NA>)
df['days']=df['days'].astype('Int64')
df['rooms']=df['rooms'].astype('Int64')
df['bathrooms']=df['bathrooms'].astype('Int64')
df['capacity']=df['capacity'].astype('Int64')

print("< Tareas completadas >")


< Tareas completadas >


In [79]:
# La columna 'province' no contiene solamente provincias
df.rename(columns={'province': 'location'}, inplace=True)
print("< Columna 'province' renombrada a 'location' >")

< Columna 'province' renombrada a 'location' >


##### 1.2.3 CREACIÓN DE NUEVAS COLUMNAS en base al Readme:
- Se necesita saber si el precio está declarado
- Determinar si el precio está en dólares
- Mostrar tasa de cambio oficial

In [80]:
# NUEVA COLUMNA: has_price -> booleano

# Si no existe un precio declarado (nulos)
df['has_price'] = ~df['price'].isna()

# Según el README, si los dígitos enteros están repetidos, el precio seguramente es falso
df['has_price'] = ~df['price'].apply(
    lambda x: (
        # no es NaN
        not pd.isna(x)
        # la parte entera tiene más de un dígito
        and len(str(int(abs(x)))) > 1
        # todos los dígitos son iguales
        and len(set(str(int(abs(x))))) == 1
    )
)

print("< 'has_price' creado exitosamente >")

< 'has_price' creado exitosamente >


In [81]:
# NUEVA COLUMNA: is_usb -> booleano

# Determina si el precio está en dólares (precio menor a 7000)
df['is_usd'] = (df['price'] < 7000) & (df['has_price'])

print("< 'is_usd' creado exitosamente >")

< 'is_usd' creado exitosamente >


In [82]:
# NUEVA COLUMNA: exchange_rate -> float

# Según el README, la tasa de cambio es 1000 en los meses de enero y diciembre
df['exchange_rate'] = df['date'].dt.month.apply(
    lambda x: 1000 if x in [12, 1] else None
)

print("< 'exchange_rate' creado exitosamente >")

< 'exchange_rate' creado exitosamente >


##### 1.2.4 VISTA PREVIA DEL DATAFRAME

In [83]:
df.sample(10)

,name,price,days,rooms,bathrooms,capacity,date,location,has_price,is_usd,exchange_rate
50808,Cabana Con Pileta Las Lomas De Miramar,100.00,1,5,3,8,2023-12-31,BUENOS AIRES,True,True,"1,000.00"
247886,Departamento Hasta 8 Personas Carlos Paz,NaN,<NA>,2,1,8,2024-01-10,CORDOBA,True,False,"1,000.00"
272206,Complejo Pinar Del Solfin De Semana Largo 9 De...,NaN,<NA>,2,2,3,2024-01-11,BUENOS AIRES,True,False,"1,000.00"
681384,Olavarria 2200 Piso 1,"160,000.00",<NA>,<NA>,1,<NA>,2024-01-30,BUENOS AIRES,True,False,"1,000.00"
101578,Chalet A 50M De Peatonal Y 2 Cuadras Del Mar,NaN,<NA>,3,2,7,2024-01-03,BUENOS AIRES,True,False,"1,000.00"
145238,Olascoaga,NaN,<NA>,2,<NA>,<NA>,2024-01-05,MENDOZA,True,False,"1,000.00"
63431,Av Colon 1900 Piso 6,30.00,<NA>,<NA>,1,<NA>,2024-01-01,BUENOS AIRES,True,True,"1,000.00"
661363,2 Amb A 2 Cuadras Del Balneario Boutique Y 6 D...,"280,000.00",7,1,1,3,2024-01-30,BUENOS AIRES,True,False,"1,000.00"
750304,Santiago Del Estero 3300,"3,000.00",<NA>,2,1,<NA>,2024-02-02,BUENOS AIRES,True,True,NaN
58324,Necochea Frente Al Mar Para 5 Personas,"210,000.00",7,1,1,4,2024-01-01,BUENOS AIRES,True,False,"1,000.00"


### 1.3 - LOAD

In [84]:

output_path = '/content/drive/MyDrive/Análisis de Datos/Dataset-Laboratorio1/alquileres_MODIFICADO_2024-02-18.csv'

# Guardar el dataframe transformado en un nuevo archivo CSV en Drive
df.to_csv(output_path, index=False, sep=',')

print(f'Archivo CSV guardado en: {output_path}')

Archivo CSV guardado en: /content/drive/MyDrive/Análisis de Datos/Dataset-Laboratorio1/alquileres_MODIFICADO_2024-02-18.csv


## 2 - **EDA**

### 2.1 - ANÁLISIS

##### 2.1.1 RESUMEN FINAL DE DATAFRAME

In [85]:
# Ajustes de cvisualización
pd.set_option('display.float_format', '{:,.2f}'.format)
print("< Hecho >")

< Hecho >


In [86]:
# Información actual del Dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1160402 entries, 0 to 1160401
Data columns (total 11 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   name           1160402 non-null  object        
 1   price          732625 non-null   float64       
 2   days           437389 non-null   Int64         
 3   rooms          1104338 non-null  Int64         
 4   bathrooms      942741 non-null   Int64         
 5   capacity       823157 non-null   Int64         
 6   date           1160402 non-null  datetime64[ns]
 7   location       1160402 non-null  category      
 8   has_price      1160402 non-null  bool          
 9   is_usd         1160402 non-null  bool          
 10  exchange_rate  709683 non-null   float64       
dtypes: Int64(4), bool(2), category(1), datetime64[ns](1), float64(2), object(1)
memory usage: 78.6+ MB


In [87]:
# Estadística básica de precios en PESOS (excluyendo aquellos que probablemente sean falsos)
df.loc[(df['has_price'] == True) & (df['is_usd'] == False), 'price'].describe()

,price
count,"505,152.00"
mean,"742,574.74"
std,"25,444,558.66"
min,"7,000.00"
25%,"35,000.00"
50%,"70,000.00"
75%,"250,000.00"
max,"1,666,666,665.00"


In [88]:
# Estadística básica de precios en DÓLARES (excluyendo aquellos que probablemente sean falsos)
df.loc[(df['has_price'] == True) & (df['is_usd'] == True), 'price'].describe()

,price
count,"209,083.00"
mean,843.00
std,938.66
min,1.00
25%,180.00
50%,600.00
75%,"1,100.00"
max,"6,600.00"


In [89]:
# DESCRIBE para PESOS para cada locación
desc_pesos = df[~df['is_usd'] & df['has_price']].groupby('location', observed=True)['price'].describe()
desc_pesos.head(20)

,count,mean,std,min,25%,50%,75%,max
location,,,,,,,,
BARILOCHE,"6,495.00","84,782.29","114,920.60","7,950.00","42,000.00","55,000.00","70,000.00","1,000,000.00"
BUENOS AIRES,"376,455.00","954,294.73","29,467,016.99","7,000.00","38,000.00","100,000.00","280,000.00","1,666,666,665.00"
CORDOBA,"48,083.00","107,730.91","191,846.30","7,000.00","30,000.00","40,000.00","85,000.00","4,500,000.00"
CORRIENTES,462.00,"71,266.23","59,052.43","15,000.00","40,000.00","50,000.00","80,000.00","370,000.00"
MAR DEL PLATA,"30,012.00","149,244.14","1,647,311.70","10,000.00","32,000.00","45,000.00","120,000.00","123,854,769.00"
MENDOZA,"20,224.00","64,699.20","1,007,523.82","7,000.00","25,000.00","30,000.00","40,000.00","100,000,000.00"
MISIONES,699.00,"153,529.33","220,276.16","7,000.00","20,000.00","35,000.00","230,000.00","880,000.00"
ROSARIO,"18,432.00","208,653.75","131,576.31","10,000.00","142,000.00","180,000.00","240,000.00","1,750,000.00"
SALTA,"3,131.00","79,262.22","154,622.13","8,000.00","24,000.00","30,000.00","50,000.00","1,200,000.00"


In [90]:
# DESCRIBE para DÓLARES para cada locación
desc_usd = df[df['is_usd'] & df['has_price']].groupby('location', observed=True)['price'].describe()
desc_usd.head(20)

,count,mean,std,min,25%,50%,75%,max
location,,,,,,,,
BARILOCHE,"2,730.00",352.74,614.71,1.00,72.00,170.00,360.00,"5,000.00"
BUENOS AIRES,"188,571.00",871.89,911.73,1.00,260.00,650.00,"1,100.00","6,600.00"
CORDOBA,"4,684.00","1,123.77","1,749.43",1.00,43.00,260.00,"1,500.00","6,500.00"
CORRIENTES,86.00,66.79,49.88,15.00,50.00,60.00,60.00,370.00
MAR DEL PLATA,"6,352.00",397.69,817.10,1.00,45.00,100.00,392.25,"6,450.00"
MENDOZA,"3,371.00",383.87,907.87,3.00,40.00,70.00,142.50,"5,700.00"
MISIONES,117.00,207.50,164.76,7.00,100.00,100.00,350.00,880.00
ROSARIO,"2,180.00",492.33,473.75,1.00,190.00,300.00,700.00,"3,000.00"
SALTA,878.00,727.84,"1,071.10",1.00,100.00,500.00,750.00,"6,500.00"


##### 2.1.2 INTERPRETACIÓN TEXTUAL en base al resumen final:

- Los precios falsos en su mayoría se encuentran en pesos y pueden distorsionar las medidas realizadas, puesto que existen valores extremos.

- Los precios en dólares parecen menos sesgados, sus valores están más concentrados y se muestra más razonable.

- Entre provincias, existen medianas muy diferentes entre sí, lo que indica una marcada diferencia regional en los precios dados.



##### 2.1.3 PREGUNTAS DE NEGOCIO

1) Los avisos en dólares, ¿dónde son más frecuentes?

In [91]:
df_tabla_evidencia = df[df['has_price']].groupby('location', observed=True)['is_usd'].mean().mul(100).sort_values(ascending=False)
df_tabla_evidencia.head(7)

,is_usd
location,
BUENOS AIRES,21.28
BARILOCHE,16.49
MAR DEL PLATA,10.93
SALTA,10.71
ROSARIO,9.96
CORRIENTES,9.29
MENDOZA,7.03


- Respuesta 1: Los avisos en dólares son más frecuentes en Buenos Aires con un 21,28% del total en esa provincia.

2) ¿Existe un sector en el que no hayan declarado precios falsos ("dummy")?

In [92]:
df_tabla_evidencia=df.groupby('location', observed=True)['has_price'].mean().mul(100).sort_values(ascending=False)
df_tabla_evidencia.head(7)

,has_price
location,
TUCUMAN,100.00
ROSARIO,99.99
MISIONES,99.90
CORRIENTES,99.78
MENDOZA,99.75
MAR DEL PLATA,99.37
CORDOBA,99.19


- Respuesta 2: Tucumán posee el 100% de sus precios declarados.

3) ¿En qué sector los avisos de alquileres son más numerosos?

In [93]:
df_tabla_evidencia=df['location'].value_counts().sort_values(ascending=False)
df_tabla_evidencia.head(7)

,count
location,
BUENOS AIRES,903199
CORDOBA,98810
MAR DEL PLATA,58507
MENDOZA,48046
ROSARIO,21890
BARILOCHE,16713
SALTA,8265


- Respuesta 3: Buenos Aires cuenta con mayor número de avisos de alquileres, con un total de 903.199.